In [ ]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
from toy_plot_helpers import ToyPlot

%config InlineBackend.figure_format='retina'

In [ ]:
import openpathsampling.toy_dynamics as toys
plot = ToyPlot()
pes = (
    toys.toy_pes.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.toy_pes.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

plot.contour_range = np.arange(-1.5, 1.0, 0.1)

topology=paths.ToyTopology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)

template = paths.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    potential_energy = 0.0, # Important to set energies dimensionless otherwise kcal/mol is default
    kinetic_energy = 0.0,
    topology=topology
)

integ = toys.toy_integrators.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'nsteps_per_frame' : 10
}

toy_eng = toys.toy_engine.ToyEngine(
    options=options,
    template=template
)
toy_eng.initialized = True

toy_eng.current_snapshot = template
paths.PathMover.engine = toy_eng

In [ ]:
plot.add_pes(pes)
fig = plot.plot()

In [ ]:
fig.axes

In [ ]:
def circle_sq(snapshot, center):
    return (snapshot.xyz[0][0]-center[0])**2 + (snapshot.xyz[0][1]-center[1])**2
    
opA = paths.CV_Function(name="opA", fcn=circle_sq, center=[-0.5, -0.5])
opB = paths.CV_Function(name="opB", fcn=circle_sq, center=[0.5, -0.5])
opC = paths.CV_Function(name="opC", fcn=circle_sq, center=[0.0, 0.4])

In [ ]:
stateA = paths.CVRangeVolume(opA, 0.0, 0.2**2)
stateB = paths.CVRangeVolume(opB, 0.0, 0.2**2)
stateC = paths.CVRangeVolume(opC, 0.0, 0.2**2)
# set up the states; interfaces
interfacesA = paths.VolumeFactory.CVRangeVolumeSet(opA, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])
interfacesB = paths.VolumeFactory.CVRangeVolumeSet(opB, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])
interfacesC = paths.VolumeFactory.CVRangeVolumeSet(opC, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])
# bootstrap out of each state to get initial trajectories
# save the initial trajectories to 3 different files.

In [ ]:
#plot.add_states([stateA, stateB, stateC])
#plot.add_interfaces(interfacesA+interfacesB+interfacesC)
#plot.plot()

In [ ]:
# the other option might be to turn this into a class FullBootstrap, but this function seems to work for now
# TODO: move this function into openpathsampling (maybe have a module bootstrap_interfaces?)
import sys
def full_bootstrap(transition, snapshot, engine, extra_interfaces=[], storage=None):
    interface0 = transition.interfaces[0]
    ensemble0 = transition.ensembles[0]
    state = transition.stateA
    first_traj_ensemble = paths.SequentialEnsemble([
        paths.OptionalEnsemble(paths.AllOutXEnsemble(state)),
        paths.AllInXEnsemble(state),
        paths.OptionalEnsemble(paths.AllOutXEnsemble(state) & paths.AllInXEnsemble(interface0)),
        paths.OptionalEnsemble(paths.AllInXEnsemble(interface0)),
        paths.AllOutXEnsemble(interface0),
        paths.OptionalEnsemble(paths.AllOutXEnsemble(state)),
        paths.SingleFrameEnsemble(paths.AllInXEnsemble(state))
    ])
    #print first_traj_ensemble #DEBUG
    engine.current_snapshot = snapshot
    engine.snapshot = snapshot
    print "Building first trajectory"
    sys.stdout.flush()
    first_traj = engine.generate(engine.current_snapshot, [first_traj_ensemble.can_append])
    print "Selecting segment"
    sys.stdout.flush()
    subtraj = ensemble0.split(first_traj)[0]
    extra_ensembles = [
        paths.TISEnsemble(transition.stateA, transition.stateB, iface, transition.orderparameter)
        for iface in extra_interfaces
    ]
    extra_shooters = [paths.OneWayShootingMover(paths.UniformSelector(), ens) for ens in extra_ensembles]
    print "Sampling " + str(len(transition.ensembles+extra_ensembles)) + " ensembles."
    bootstrap = paths.Bootstrapping(
        storage=storage,
        ensembles=transition.ensembles+extra_ensembles,
        movers=transition.movers['shooting']+extra_shooters,
        trajectory=subtraj
    )
    print "Beginning bootstrapping"
    while len(bootstrap.globalstate) < len(transition.ensembles+extra_ensembles):
        print len(bootstrap.globalstate), "samples so far"
        sys.stdout.flush()
        bootstrap.run(20)
    print len(bootstrap.globalstate), "samples so far (DONE!)"
    return bootstrap.globalstate


In [ ]:
mstis = paths.MSTISNetwork([
    (stateA, interfacesA, "A", opA),
    (stateB, interfacesB, "B", opB),
    (stateC, interfacesC, "C", opC)
])

In [ ]:
print mstis

In [ ]:
for t in mstis.sampling_transitions:
    print t

In [ ]:
print mstis.transitions

### Bootstrap to fill all interfaces

In [ ]:
initA = paths.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[1.0,0.0]]),
)
gsA = full_bootstrap(mstis.from_state[stateA], initA, toy_eng, extra_interfaces=[interfacesA[-1]])

In [ ]:
initB = paths.Snapshot(
    coordinates=np.array([[0.5, -0.5]]), 
    velocities=np.array([[-1.0,0.0]]),
)
gsB = full_bootstrap(mstis.from_state[stateB], initB, toy_eng, extra_interfaces=[interfacesB[-1]])

In [ ]:
initC = paths.Snapshot(
    coordinates=np.array([[0.0, 0.4]]), 
    velocities=np.array([[0.0,-0.5]]),
)
gsC = []
gsC = full_bootstrap(mstis.from_state[stateC], initC, toy_eng, extra_interfaces=[interfacesC[-1]])

In [ ]:
plot.plot([s.trajectory for s in gsA]+[s.trajectory for s in gsB]+[s.trajectory for s in gsC]);

In [ ]:
outer_sset = paths.SampleSet([gs.samples[-1] for gs in [gsA, gsB, gsC]])
sset = paths.SampleSet.relabel_replicas_per_ensemble([
        paths.SampleSet(gsA.samples[:-1]), 
        paths.SampleSet(gsB.samples[:-1]),
        paths.SampleSet(gsC.samples[:-1])
    ])
# TODO: the better approach is to adjust relabel_replicas_per_ensemble so that it drops extra ensembles 
# ... not sure what the current behavior is

In [ ]:
msouter_sample = paths.Sample(
    replica=len(sset),
    trajectory=outer_sset.samples[0].trajectory,
    ensemble=mstis.ms_outers[0]
)
sset = sset.apply_samples([msouter_sample])
minus_count = 0
minus_samples = []
for retis in mstis.from_state.values():
    minus_count -= 1
    minus_samples.append(retis.minus_ensemble.populate_minus_ensemble(
        partial_traj=sset[retis.ensembles[0]].trajectory,
        minus_replica_id=minus_count,
        engine=toy_eng
    ))
sset = sset.apply_samples(minus_samples)
# TODO: add samples for the minus interfaces, too

In [ ]:
storage = paths.storage.Storage("mstis.nc", "w", template=template)
storage.save(mstis)
mstis_calc = paths.PathSampling(
    storage=storage,
    engine=toy_eng,
    move_scheme=mstis.default_schemes(),
    globalstate=sset
)

In [ ]:
xval = paths.CV_Function("xval", lambda snap : snap.xyz[0][0])
yval = paths.CV_Function("yval", lambda snap : snap.xyz[0][1])
#mstis_calc.live_visualization = paths.LiveVisualization(mstis, xval, yval, [-1.0, 1.0], [-1.0, 1.0])
background = ToyPlot()
background.contour_range = np.arange(-1.5, 1.0, 0.1)
background.add_pes(pes)
#mstis_calc.live_visualization.background = background.plot()

In [ ]:
mstis_calc.run(200)

In [ ]:
%%time
storage.sync()